## PDF Yüklemesinin Gerçekleştirilmesi

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Yazılımcıların Bilmesi Gereken Algoritmalar.pdf")
data = loader.load()

In [5]:
len(data)

104

## Veriyi Parçalara Ayırma (Chunking İşlemi)

* Belgeyi daha küçük parçalara ayırarak, doğru bilgiyi bu sayede daha kolay bir şekilde ulaşmaktır.
  
* Chunk size, genelde 512 ile 2000 arasında seçilir.

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)

docs = text_splitter.split_documents(data)

print("Total:", len(docs))

Total: 261


In [19]:
# doc içindeki detay bilgilerin değerlerini görmek için, dokumana herhangi bir indeks verebiliriz.
# page_content : pdf içerisindeki asıl metni tutar.

docs[5]

Document(metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-04-25T12:07:00+00:00', 'title': 'ALGORİTMALAR E-KİTAP', 'moddate': '2025-04-25T12:06:50+00:00', 'keywords': 'DAGEeIaHfOY,BAFSm9mcL9Q,0', 'author': 'Enes Toy', 'source': 'Yazılımcıların Bilmesi Gereken Algoritmalar.pdf', 'total_pages': 104, 'page': 4, 'page_label': '5'}, page_content='Sevgili Okuyucu,\nYazılım geliştirme, yalnızca kod yazmaktan ibaret değil; akıllı ve\nverimli algoritmalar, modern iş dünyasında süreçleri optimize etmenin\nve rekabet avantajı sağlamanın anahtarıdır. İster finans, sağlık, ister\ne-ticaret sektöründe olun, güçlü algoritmalar sayesinde karmaşık\nproblemleri çözebilir ve daha etkin çözümler üretebilirsiniz.\nBu e-kitap, yazılım geliştiricilerin iş dünyasında ihtiyaç duyduğu temel\nve ileri düzey algoritmaları kapsamlı ve anlaşılır bir şekilde ele alıyor.\nAmacım, yalnızca teorik bilgiler vermek değil, aynı zamanda bu\nalgoritmaların gerçek dünya senaryolarında nasıl uygulandığı

## Google Generative AI Embedding Oluşturmak

* Amaç, text verimizi sayısal bir veriye çevirmeye çalışmaktır.

In [23]:
from langchain_chroma import Chroma
# Embedding oluşturabilmek için GoogleGenerativeAIEmbeddings paketini yükledik.
# embbedding: metni sayısal bir vektöre dönüştürmek
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv # env içerisindeki key'i okur ve modeli otomatik olarak kullanılabilir hale geliriz.
load_dotenv()

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

# Burada verdiğimiz text ifadeyi vektöre çevirebilecek mi diye test ediyoruz. 
vector = embeddings.embed_query("hello, world!") 
vector[:5] # vektörün yalnızca ilk 5 elemanını bize gösterir.

[0.014134909026324749,
 -0.022324152290821075,
 -0.054603420197963715,
 -0.006284549366682768,
 -0.03392402455210686]

## ChromaDB Üzerine Kayıt İşlemi

* Amaç, metinleri vektör formatında saklamaktır.
* Bunun için metinleri vektör formatına çevirir ve ChromaDB içerisine kaydedilir.
* Gün sonundaki amacımız, vektör veritabanında en alakalı belgeleri bulabilmektir. Yani, similarity tarafındaki en benzer alanları bulabilmektir.

In [36]:
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)

# search_type'da "similarity" kullandık çünkü en alakalı belgeleri bulmak istiyoruz.
# search_kwargs tarafında vereceğimiz değer, bize kaç sonuç döndüreceğimizi söyler.
# Örneğin: k= 10 ise, en alakalı bize 10 belgeyi geri döndürür.
# Burada sorulan soruya uygun olarak, en iyi belgeleri çekmesini sağlarız. (retriever yaparız.)
retriever = vectorstore.as_retriever(
    search_type="similarity", search_kwargs={"k": 10}
)

In [54]:
retrieved_docs = retriever.invoke("Bubble Sort Nedir?")

In [56]:
len(retrieved_docs)

10

In [58]:
print(retrieved_docs[5].page_content)

bölme) tekrarlanır.
Bu  şlem, her b r alt d z  tek elemanlı hale gelene kadar devam eder.
3.B rleşt rme:
Alt d z ler sıralandıktan sonra, tek tek b rleşt r l r ve n ha  sıralı d z  elde ed l r.
Adım Adım Örnek
Örnek d z : [10, 80, 30, 90, 40, 50, 70]
1.P vot Seç m  ve Bölme:
İlk adımda, p vot olarak 70 seç l r.
70'ten küçük olan elemanlar: [10, 30, 40, 50]
70'ten büyük olan elemanlar: [80, 90]
P vot yer nde: [10, 30, 40, 50, 70, 80, 90]
2.Alt D z ler n Sıralanması:
Sol taraf: [10, 30, 40, 50] →  P vot 50 seç l r, sıralama yapılır.
Sağ taraf: [80, 90] →  P vot 90 seç l r, sıralama yapılır.
3.Sonuç:
N ha  sıralı d z : [10, 30, 40, 50, 70, 80, 90]
Qu ck Sort, en kötü durumda O(n ² ) zaman karmaşıklığına sah p olab l r ancak genell kle
O(n log n) performans göster r.
Avantajları
✅  Hızlıdır: Ortalama durumda O(n log n) zaman karmaşıklığıyla oldukça hızlıdır.
✅  Yer nde sıralama yapar: Ekstra bellek kullanmaz.
Dezavantajları


## Google Gemini API Yapısını Kullanarak LLM Invoke İşlemleri

* Temperature, modelin yanıtlarındaki rastgeleliği ve yaratıcılığı belirler.

* Temperature Değer Aralığı:

    * **Düşük Değerler(0.1-0.3):** Daha kesin ve daha tutarlı cevaplar vermemizi sağlayabilir. Model, daha tahmin edilebilir hale gelir.
    * **Orta Değerler (0.4-0.7):**  Hem mantıklı hem de yaratıcı cevaplar verebiliyor.
    * **Yüksek Değerler(0.7-1.0):** Daha rastgele ve yaratıcı cevaplar ve bazen de belki tutarsız cevaplar verebilir.

      


*  Max_tokens, modelin ürettiği yanıtın maximum uzunluğudur. Böylece, çok uzun ve gereksiz yanıtların önüne geçip, API kullanımı tarafında da maliyeti düşürmüş oluruz.

* Token: Kelime parçacığı

In [67]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    temperature=0.3,
    max_tokens= 500
)

In [75]:
# Langchain amaç, bir dil zinciri oluşturmak
# Gün sonunda retriever + LLM + Prompt'un birleştiği zinciri birleştiren kısım
# Prompt Template = System prompt + User prompt
# combine_documents => Dokümanların zincirini oluşturduğum kısım

from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In [77]:
system_prompt = ("You are an assistant for question-answering tasks."
                 "Use the following pieces of retrieved context to answer."
                 "If you don't know the answer, say that you don't know."
                 "Use three sentences maximum and keep the answer corrects."
                 "\n\n"
                 "{context}"    # context: retriever'dan gelecek değere işaret ediyor.
                 )

In [79]:
prompt = ChatPromptTemplate(
     [
        ("system", system_prompt),
        ("human", "{input}")
         
     ]
)

## Soru-Cevap Üretme Zinciri Oluşturma (LLM + Prompt)

In [82]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)

## RAG Zinciri Oluşturma (RAG + LLM Entegrasyonun Gerçekleşmesi)

*  **retriever**: Vektör veritabanındaki ilgili belgeleri bulur ve question-answer chain içerisine aktarır.
*  **question_answer_chain**: LLM'de bu belgeleri alarak bir yanıt üretir. Yanıt da, "response" değişkeninde saklanır.

In [93]:
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

## Kullanıcı Sorgusunu Çalıştırarak Cevap Üretme İşlemi

In [96]:
response = rag_chain.invoke({"input":"Bubble Sort Nedir?"})

In [98]:
print(response)

{'input': 'Bubble Sort Nedir?', 'context': [Document(id='f8020f85-ec9f-456c-82b0-f0d2b24ea4c2', metadata={'author': 'Enes Toy', 'creationdate': '2025-04-25T12:07:00+00:00', 'creator': 'Canva', 'keywords': 'DAGEeIaHfOY,BAFSm9mcL9Q,0', 'moddate': '2025-04-25T12:06:50+00:00', 'page': 5, 'page_label': '6', 'producer': 'Canva', 'source': 'Yazılımcıların Bilmesi Gereken Algoritmalar.pdf', 'title': 'ALGORİTMALAR E-KİTAP', 'total_pages': 104}, page_content='55\nBubble Sort Algor\x00tması\nBubble Sort, sıralama algor\x00tmalarından b\x00r\x00d\x00r ve oldukça bas\x00t b\x00r mantığa dayanır. Bu\nalgor\x00tma, l\x00stedek\x00 her öğey\x00 b\x00rb\x00r\x00yle karşılaştırır ve sırasıyla yer değ\x00şt\x00r\x00r. B\x00r d\x00z\x00\n\x00ç\x00nde sıralama yaparken, her geç\x00şte en büyük (veya en küçük) öğe sona "baloncuk"\ng\x00b\x00 yüksel\x00r. Bu \x00şlem, tüm öğeler sıralanana kadar devam eder.\nÇalışma Prens\x00b\x00\n1.L\x00stedek\x00 öğeler\x00 b\x00rb\x00r\x00yle karşılaştırır.\n2.Eğer b\x00

In [100]:
print(response['answer'])

Bubble Sort, listedeki her öğeyi birbiriyle karşılaştırıp sırasıyla yer değiştiren basit bir sıralama algoritmasıdır. Bir dizi içinde sıralama yaparken, her geçişte en büyük (veya en küçük) öğe sona doğru yükselir. Bu işlem, tüm öğeler sıralanana kadar devam eder.
